<a href="https://colab.research.google.com/github/gabrielhozana/Sentiment-Analysis-with-Pyspark-MLlib/blob/main/Sentiment_Analysis_with_Pyspark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pandas to Visualization Dataset

In [1]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Train1.csv")

In [2]:
data.head()

,label,reviewTitle,reviewText
0,2,Stuning even for the non-gamer,NaN
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,NaN,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,NaN
4,2,NaN,"If you've played the game, you know how divine..."


# Pyspark is ready to use

# Debungging Small Dataset with null

In [3]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 38.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=8a189405468982b488cf3cba4b8be1be9ef0edc35132ea4dc0ccc9877c081afa
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
from pyspark.sql import Row
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
# documents = sc.dataframe("/content/drive/MyDrive/train.csv").map(lambda line: line.split(" "))
# save data https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html
df = spark.read.load("/content/drive/MyDrive/Train1.csv",
                     format="csv", sep=",", inferSchema="true", header="true")
df.show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|Stuning even for ...|                null|
|    2|The best soundtra...|I'm reading a lot...|
|    2|                null|"This soundtrack ...|
|    2|Excellent Soundtrack|                null|
|    2|                null|If you've played ...|
|    2|an absolute maste...|                null|
|    1|        Buyer beware|                null|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
+-----+--------------------+--------------------+



In [5]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+-----------+----------+
|label|reviewTitle|reviewText|
+-----+-----------+----------+
|    0|          2|         4|
+-----+-----------+----------+



In [6]:
df.na.drop().show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|The best soundtra...|I'm reading a lot...|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
+-----+--------------------+--------------------+



In [7]:
df.show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|Stuning even for ...|                null|
|    2|The best soundtra...|I'm reading a lot...|
|    2|                null|"This soundtrack ...|
|    2|Excellent Soundtrack|                null|
|    2|                null|If you've played ...|
|    2|an absolute maste...|                null|
|    1|        Buyer beware|                null|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
+-----+--------------------+--------------------+



In [8]:
df2 = df.dropna(subset=('label','reviewTitle','reviewText'))

df2.show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|The best soundtra...|I'm reading a lot...|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
+-----+--------------------+--------------------+



In [9]:
def tfIdf(df):
    """ This fucntion takes the text data and converts it into a term frequency-Inverse Document Frequency vector

    parameter: 
    returns: dataframe with tf-idf vectors

    """
    # Carrying out the Tokenization of the text documents (splitting into words)

    tokenizer = Tokenizer(inputCol="reviewText", outputCol="tokenised_text")
    tokensDf = tokenizer.transform(df)

    # Carrying out the StopWords Removal for TF-IDF
    stopwordsremover=StopWordsRemover(inputCol='tokenised_text',outputCol='words')
    swremovedDf= stopwordsremover.transform(tokensDf)

    # Creating Term Frequency Vector for each word
    cv=CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=3, minDF=2.0)
    cvModel=cv.fit(swremovedDf)
    tfDf=cvModel.transform(swremovedDf)

    # Carrying out Inverse Document Frequency on the TF data
    idf=IDF(inputCol="tf_features", outputCol="tf-idf_features")
    idfModel = idf.fit(tfDf)
    tfidfDf = idfModel.transform(tfDf)

    tfidfDf.cache().count()

    return tfidfDf


tfidfDf=tfIdf(df2)

In [10]:
tfidfDf.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|         reviewTitle|          reviewText|      tokenised_text|               words|         tf_features|     tf-idf_features|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    2|The best soundtra...|I'm reading a lot...|[i'm, reading, a,...|[reading, lot, re...|       (3,[2],[1.0])|(3,[2],[0.1823215...|
|    2|      Glorious story|I loved Whisper o...|[i, loved, whispe...|[loved, whisper, ...|(3,[0,1,2],[3.0,1...|(3,[0,1,2],[1.216...|
|    2|    A FIVE STAR BOOK|I just finished r...|[i, just, finishe...|[finished, readin...| (3,[0,2],[1.0,1.0])|(3,[0,2],[0.40546...|
|    2|Whispers of the W...|This was a easy t...|[this, was, a, ea...|[easy, read, book...|(3,[0,1,2],[2.0,4...|(3,[0,1,2],[0.810...|
|    1|          The Worst!|A complete waste ...|[a, complete,

# Using Pure Dataset 

In [11]:
df3 = spark.read.load("/content/drive/MyDrive/train.csv",
                     format="csv", sep=",", inferSchema="true", header="true")
df3.show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|Stuning even for ...|This sound track ...|
|    2|The best soundtra...|I'm reading a lot...|
|    2|            Amazing!|"This soundtrack ...|
|    2|Excellent Soundtrack|I truly like this...|
|    2|Remember, Pull Yo...|If you've played ...|
|    2|an absolute maste...|I am quite sure a...|
|    1|        Buyer beware|"This is a self-p...|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
|    2|          Great book|This was a great ...|
|    2|          Great Read|I thought this bo...|
|    1|           Oh please|I guess you have ...|
|    1|Awful beyond belief!|"I feel I have to...|
|    1|Don't try to fool...|It's glaringly ob...|
|    2|A romantic zen ba...|"When you hear fo...|


In [ ]:
df3.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:36459)
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: ignored

In [ ]:
df4 = df3.dropna(subset=('label','reviewTitle','reviewText'))
df4.show()

+-----+--------------------+--------------------+
|label|         reviewTitle|          reviewText|
+-----+--------------------+--------------------+
|    2|Stuning even for ...|This sound track ...|
|    2|The best soundtra...|I'm reading a lot...|
|    2|            Amazing!|"This soundtrack ...|
|    2|Excellent Soundtrack|I truly like this...|
|    2|Remember, Pull Yo...|If you've played ...|
|    2|an absolute maste...|I am quite sure a...|
|    1|        Buyer beware|"This is a self-p...|
|    2|      Glorious story|I loved Whisper o...|
|    2|    A FIVE STAR BOOK|I just finished r...|
|    2|Whispers of the W...|This was a easy t...|
|    1|          The Worst!|A complete waste ...|
|    2|          Great book|This was a great ...|
|    2|          Great Read|I thought this bo...|
|    1|           Oh please|I guess you have ...|
|    1|Awful beyond belief!|"I feel I have to...|
|    1|Don't try to fool...|It's glaringly ob...|
|    2|A romantic zen ba...|"When you hear fo...|


In [ ]:
df4.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

In [ ]:
tfidfDf2=tfIdf(df4)
tfidfDf2.show()

Py4JJavaError: ignored